# Multi-Agent Travel Planner with AutoGen

This notebook will guide you through building a collaborative travel planning system using multiple AI agents. Each agent has a different role, and together they help create a personalized travel plan.

We'll go step by step:
1. Set up the environment and import libraries
2. Load API keys and configure the model
3. Define specialized agents
4. Set up group chat for agent collaboration
5. Run a travel planning task and view the results


## 1. Import Required Libraries

First, let's import the libraries we need. These include AutoGen modules for agents and group chat, as well as dotenv for loading environment variables.

In [1]:
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

## 2. Load API Keys and Configure Model

We'll load our API keys from a `.env` file and set up the OpenAI model client. This allows our agents to use GPT-4o-mini for generating responses.

In [2]:
load_dotenv()

model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

## 3. Define Specialized Agents

We'll create three agents, each with a different role:
- **Planner Agent**: Designs the travel itinerary.
- **Budget Agent**: Checks if the plan fits the budget and suggests savings.
- **Activity Agent**: Recommends places and activities to do.

Each agent uses the same model client but has a unique system message describing its expertise.

In [ ]:
planner_agent = AssistantAgent(
    name="planner_agent",
    model_client=model_client,
    description="Creates personalized travel plans.",
    system_message="You are a travel planner. Suggest a detailed itinerary based on destination, budget, and preferences."
)

budget_agent = AssistantAgent(
    name="budget_agent",
    model_client=model_client,
    description="Provides budgeting advice for travel.",
    system_message="You are a budgeting expert. Ensure the travel plan fits within the user's budget and suggest cost-saving tips."
)

activity_agent = AssistantAgent(
    name="activity_agent",
    model_client=model_client,
    description="Recommends activities and places to visit.",
    system_message="You are an activity expert. Recommend attractions, activities, and experiences for the destination."
)

## 4. Set Up Group Chat for Collaboration

Now we'll create a group chat where all agents can work together. We'll use a round-robin approach so each agent gets a turn to contribute. The chat will end when someone mentions 'TERMINATE'.

In [ ]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, budget_agent, activity_agent],
    termination_condition=termination
)

## 5. Run the Travel Planning Task

Let's ask our agents to plan a 5-day vacation to Japan with a budget of $2500. The agents will collaborate and share their expertise to create a complete plan. The results will be shown in the console.

In [ ]:
async def main():
    await Console(group_chat.run_stream(task="I want a 5-day vacation to Japan under $2500."))
    await model_client.close()

asyncio.run(main())